In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow.keras.callbacks import TensorBoard
print(tf.__version__)

In [ ]:
tensorboard = TensorBoard(
  log_dir='.\logs',
  histogram_freq=1,
  write_images=True
)
keras_callbacks = [
  tensorboard
]

# Loading Data

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')

# Splitting Train Data into Test and Train Datasets

In [ ]:
train_dataset = train_data.sample(frac=0.8, random_state=0)
test_dataset = train_data.drop(train_dataset.index)

# EDA

In [ ]:
features = ['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5']


In [ ]:
train_dataset.describe().T

In [ ]:
sns.heatmap(train_dataset.corr())

# Separating Features and labels

In [ ]:
train_features = train_data.copy()
test_features = test_dataset.copy()

train_labels = pd.DataFrame([train_features.pop(x) for x in ['date_time','target_carbon_monoxide','target_benzene','target_nitrogen_oxides']]).T
test_labels = pd.DataFrame([test_features.pop(x) for x in ['date_time','target_carbon_monoxide','target_benzene','target_nitrogen_oxides']]).T


# Model 

In [ ]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features[features]))

In [ ]:
model = keras.Sequential([ normalizer, 
                          layers.Dense(16, activation='relu'), 
                          layers.Dense(16, activation='relu'),
                          layers.Dense(3)])
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.001))


In [ ]:
history = model.fit(train_features[features], 
                    np.asarray(train_labels[['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']]).astype('float32'),
                    validation_split=0.2, verbose=0, epochs=100, callbacks=keras_callbacks)


In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 50000])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)
  plt.show()

In [ ]:
plot_loss(history)

# Testing Model Predictions

In [ ]:
test_predictions = model.predict(test_features[features])

In [ ]:
def r_squared(y, f):
    ss_tot = np.sum((y - (np.sum(y)/len(y)))**2)
    ss_res = np.sum((y - f)**2)
    return 1 - ss_res/ss_tot

In [ ]:
r_squared(test_labels['target_carbon_monoxide'],test_predictions[:,0])

In [ ]:
r_squared(test_labels['target_benzene'],test_predictions[:,1])

In [ ]:
r_squared(test_labels['target_nitrogen_oxides'],test_predictions[:,2])

# Making Predictions

In [ ]:
test_labels = pd.DataFrame(test_data.pop('date_time'))

In [ ]:
pred = model.predict(test_data[features])
test_labels['target_carbon_monoxide'] = pred[:,0]
test_labels['target_benzene'] = pred[:,1]
test_labels['target_nitrogen_oxides'] = pred[:,2]

test_labels.to_csv('submission.csv',index = False)